<a href="https://colab.research.google.com/github/eggtartplus/nlp/blob/main/dl_ass1EX2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn
from torchvision.models import vgg16
from torch.utils.data import DataLoader

import os

In [2]:
!cd /content
!pwd
from google.colab import drive
drive.mount('/content/drive')
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version
# Check PyTorch installation
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

/content
Mounted at /content/drive
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

1.13.1+cu116
True


In [3]:
!cd '/content/drive/MyDrive/Colab Notebooks/EX2'
# This is a quite simple CNN with 3 convolutional layers
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        # the first layer of the CNN
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, 7, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )
        # the second layer of the CNN
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, 7, stride=2, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )
        self.conv3 = nn.Sequential(
            # GRADED FUNCTION: Please define the third layer of the CNN. 
            # Conv2D with 128 5x5 filters and stride of 2
            # ReLU
            # MaxPool2d with 2x2 filters and stride of 2
            ### START SOLUTION HERE ###
            nn.Conv2d(128, 128, 5, stride=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
            ### END SOLUTION HERE ###
        )
        self.classifier = nn.Sequential(
            # GRADED FUNCTION: Please define the classifier
            # Linear with input size of 128 x width? x height? and output size of 4096
            # ReLU
            # Linear with input size of 4096 and output size of 4096
            # ReLU
            # Linear with input size of 4096 and output size of number of classes
            ### START SOLUTION HERE ###
            nn.Linear(4608,4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096,4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096,5)
            ### END SOLUTION HERE ###
        )
    def forward(self, x):
        # print(x.size())
        x = self.conv1(x)
        # print(x.size())
        x = self.conv2(x)
        # print(x.size())
        x = self.conv3(x)
        # print(x.size())
        # GRADED FUNCTION: Flatten Layer
        ### START SOLUTION HERE ###
        x = torch.flatten(x, 1)
        # print(x.size())
        ### END SOLUTION HERE ###
        x = self.classifier(x)
        return x

def train(train_loader, model, loss_fn, optimizer, device):
    for i, (image, annotation) in enumerate(train_loader):
        # move data to the same device as model
        image = image.to(device)
        annotation = annotation.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward and compute prediction error
        output = model(image)
        loss = loss_fn(output, annotation)
        # backward + optimize
        loss.backward()
        optimizer.step()

        # print statistics
        if i % 20 == 0:    # print every 20 iterates
            print(f'iterate {i + 1}: loss={loss:>7f}')

def val(val_loader, model, device):
    # switch to evaluate mode
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (image, annotation) in enumerate(val_loader):
            # move data to the same device as model
            image = image.to(device)
            annotation = annotation.to(device)

            # network forward
            output = model(image)

            # for compute accuracy
            _, predicted = torch.max(output.data, 1)
            total += annotation.size(0)
            correct += (predicted == annotation).sum().item()

    # GRADED FUNCTION: calculate the accuracy using variables before
    # use variable named 'acc' to store the accuracy
    ### START SOLUTION HERE ###
    acc = correct/total
    ### END SOLUTION HERE ###
    print(f'total val accuracy: {100 * acc:>2f} %')
    return acc

In [5]:
os.chdir("/content/drive/My Drive/Colab Notebooks/mmclassification/data")
if __name__ == '__main__':
    # define image transform
    transform = transforms.Compose([
                    transforms.RandomResizedCrop(224),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])
    batch_size = 64

    # loda data
    traindir = os.path.join('flower_dataset', 'training_set')
    valdir = os.path.join('flower_dataset', 'val_set')    
    # GRADED FUNCTION: define train_loader and val_loader
    ### START SOLUTION HERE ###
    trainset = datasets.ImageFolder(traindir, transform)
    valset = datasets.ImageFolder(valdir, transform)
    train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(valset, batch_size=batch_size, shuffle=False)
    ### END SOLUTION HERE ###

    # device used to train
    device = torch.device("cuda:0")
    # GRADED FUNCTION: define a SimpleCNN model and move it to the device
    # use variable named 'model' to store this model
    ### START SOLUTION HERE ###
    model = SimpleCNN(num_classes=5).to(device)
    ### END SOLUTION HERE ###

    # Classification Cross-Entropy loss 
    loss_fn = nn.CrossEntropyLoss()

    # GRADED FUNCTION: Please define the optimizer as SGD with lr=0.05, momentum=0.9, weight_decay=0.0001
    ### START SOLUTION HERE ###
    optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=0.0001)
    ### END SOLUTION HERE ###

    # GRADED FUNCTION: Please define the scheduler
    # the learning rate will decay 0.05 every 5 steps
    ### START SOLUTION HERE ###
    scheduler = StepLR(optimizer, step_size=5, gamma=0.05)
    ### END SOLUTION HERE ###

    # create model save path
    os.makedirs('work_dir', exist_ok=True)

    max_acc = -float('inf')
    for epoch in range(10):
        print('-' * 30, 'epoch', epoch + 1, '-' * 30)

        # train
        train(train_loader, model, loss_fn, optimizer, device)
        print('lr: {}'.format(optimizer.param_groups[0]['lr']))

        # validation
        acc = val(val_loader, model, device)

        # save best model
        if acc > max_acc:
            pt_path = os.path.join('work_dir', 'best.pt')
            torch.save(model.state_dict(), pt_path)
            print('save model')

        # decay learning rate
        scheduler.step()
    print('Finished Training')

------------------------------ epoch 1 ------------------------------
iterate 1: loss=1.605122
iterate 21: loss=1.301341
lr: 0.05
total val accuracy: 42.105263 %
save model
------------------------------ epoch 2 ------------------------------
iterate 1: loss=1.357102
iterate 21: loss=1.459450
lr: 0.05
total val accuracy: 39.122807 %
save model
------------------------------ epoch 3 ------------------------------
iterate 1: loss=1.636499
iterate 21: loss=1.195161
lr: 0.05
total val accuracy: 38.421053 %
save model
------------------------------ epoch 4 ------------------------------
iterate 1: loss=1.400517
iterate 21: loss=1.256011
lr: 0.05
total val accuracy: 38.947368 %
save model
------------------------------ epoch 5 ------------------------------
iterate 1: loss=1.378858
iterate 21: loss=1.384409
lr: 0.05
total val accuracy: 53.859649 %
save model
------------------------------ epoch 6 ------------------------------
iterate 1: loss=1.221678
iterate 21: loss=1.346022
lr: 0.00250000